In [1]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import numpy as np
import pandas as pd

from char_extractor import CharExtractor
from label_classifier import LabelClassifier

In [2]:
train = pd.read_csv('./en_train.csv', sep=',', encoding='utf8')
train.dropna(axis=0, how='any', inplace=True)
train['before'] = train.before.astype(unicode)
train['after'] = train.after.astype(unicode)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9918390 entries, 0 to 9918440
Data columns (total 5 columns):
sentence_id    int64
token_id       int64
class          object
before         object
after          object
dtypes: int64(2), object(3)
memory usage: 454.0+ MB


In [3]:

char_extractor = CharExtractor.load_from_file('./lc_params.json')
validation_data = train['class'].values
input_data = train['before'].values

In [4]:

tf.reset_default_graph()

model = LabelClassifier(char_extractor._embedding_size, char_extractor._embeddings, char_extractor._char_to_id,
                        char_extractor._start_id, char_extractor._end_id, char_extractor._label_one_hot,
                        len(char_extractor._label_one_hot), char_extractor._id_to_label,
                        char_extractor._label_to_id, state_size=256, num_layers=8)
model.save_params('./lc_model.json')

result = model.classify('./lc_classifier', input_data[:30000], 10000)



/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from ./lc_classifier/best.chkp
number of iterations:  3
processing batch:  0
processing batch:  1
processing batch:  2


In [5]:
errors = []
for i in range(len(result)):
    if result[i] != validation_data[i]:
        errors.append((input_data[i], validation_data[i], result[i]))

print len(errors)

259


In [17]:
import sys

for error in errors[:100]:
    sys.stdout.write('"{0}" >> {1} = {2}\n'.format(error[0].encode('utf-8'), error[1], error[2]))

"ALCS" >> LETTERS = PLAIN
"-" >> VERBATIM = PLAIN
"LL" >> PLAIN = LETTERS
"Aceh" >> LETTERS = PLAIN
"Aceh" >> LETTERS = PLAIN
"Lviv" >> LETTERS = PLAIN
"Ukh" >> LETTERS = PLAIN
"FRELIMO" >> PLAIN = LETTERS
"POMZ" >> LETTERS = PLAIN
"Boev" >> LETTERS = PLAIN
"-" >> VERBATIM = PLAIN
"25-08-2015" >> DATE = TELEPHONE
"POLARCAT" >> LETTERS = PLAIN
"M" >> LETTERS = PLAIN
"DoD" >> LETTERS = PLAIN
"Revd" >> LETTERS = PLAIN
"VIDEO" >> PLAIN = LETTERS
"-" >> VERBATIM = PLAIN
"OL" >> LETTERS = PLAIN
"MUV" >> LETTERS = PLAIN
"SUV" >> LETTERS = PLAIN
"UA" >> PLAIN = LETTERS
"CO" >> LETTERS = VERBATIM
"DOH" >> PLAIN = LETTERS
"SHU" >> PLAIN = LETTERS
"F" >> LETTERS = PLAIN
"200" >> DIGIT = CARDINAL
"X" >> LETTERS = PLAIN
"PIALAT" >> LETTERS = PLAIN
"GETS" >> PLAIN = LETTERS
"b" >> LETTERS = PLAIN
"8 a.m." >> TIME = DATE
"XIX" >> CARDINAL = PLAIN
"G" >> PLAIN = LETTERS
"-" >> VERBATIM = PLAIN
"-" >> VERBATIM = PLAIN
"Wim" >> LETTERS = PLAIN
"INTELLIGENCE" >> PLAIN = LETTERS
"-" >> VERBATIM = PLAIN
"7

In [5]:
validation_data = train['class'].values
input_data = train['before'].values

tf.reset_default_graph()

model = LabelClassifier.load_params('./lc_model.json')


sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
saver.restore(sess, './lc_classifier/best.chkp')


result = model.generate(sess, input_data[:10000])
errors = []
for i in range(len(result)):
    if validation_data[i] != result[i][0]:
        print '"{0}" >> {1} = {2} {3} {4}'.format(input_data[i].encode('utf-8'), validation_data[i], result[i][0], result[i][1], result[i][2])
        errors.append((input_data[i], validation_data[i], result[i]))


INFO:tensorflow:Restoring parameters from ./lc_classifier/best.chkp
"ALCS" >> LETTERS = PLAIN LETTERS VERBATIM
"-" >> VERBATIM = PLAIN VERBATIM PUNCT
"LL" >> PLAIN = LETTERS PLAIN VERBATIM
"Aceh" >> LETTERS = PLAIN LETTERS VERBATIM
"Aceh" >> LETTERS = PLAIN LETTERS VERBATIM
"Lviv" >> LETTERS = PLAIN LETTERS VERBATIM
"Ukh" >> LETTERS = PLAIN LETTERS VERBATIM
"FRELIMO" >> PLAIN = LETTERS PLAIN VERBATIM
"POMZ" >> LETTERS = PLAIN LETTERS VERBATIM
"Boev" >> LETTERS = PLAIN LETTERS VERBATIM
"-" >> VERBATIM = PLAIN VERBATIM PUNCT
"25-08-2015" >> DATE = TELEPHONE DATE MEASURE
"POLARCAT" >> LETTERS = PLAIN LETTERS VERBATIM
"M" >> LETTERS = PLAIN LETTERS VERBATIM
"DoD" >> LETTERS = PLAIN LETTERS VERBATIM
"Revd" >> LETTERS = PLAIN LETTERS VERBATIM
"VIDEO" >> PLAIN = LETTERS PLAIN ORDINAL
"-" >> VERBATIM = PLAIN VERBATIM PUNCT
"OL" >> LETTERS = PLAIN LETTERS ORDINAL
"MUV" >> LETTERS = PLAIN LETTERS VERBATIM
"SUV" >> LETTERS = PLAIN LETTERS VERBATIM
"UA" >> PLAIN = LETTERS VERBATIM PLAIN
"CO" >> LE

In [4]:
print len(errors)

for error in errors[:100]:
    print '"{0}" >> {1} = {2} {3} {4}\n'.format(error[0].encode('utf-8'), error[1], error[2][0], error[2][2], error[2][2])

0
